In [1]:
# create datacard for 2 tag

import ROOT as rt
import csv
import re
import sys
import collections
import os

from collections import OrderedDict
import uproot
import pandas as pd
import math
import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append('/storage/af/user/christiw/gpu/christiw/llp/delayed_jet_analyzer/lib/')
sys.path.append('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/python/')
from helper import make_datacard_2sig, make_datacard_2tag, weight_calc
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from helper_functions import deltaR, deltaPhi

import CMS_lumi, tdrstyle
tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0


print(sys.version)

Welcome to JupyROOT 6.18/02
3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [2]:
fpath =OrderedDict()
tree = OrderedDict()
mass = [15, 40, 55]


old_ctau = {
    'STodd_ms3p0':[100,500],
    'SToEE_ms0p4':[10,50],
    'SToGammaGamma_ms0p4':[10,50],
    'SToKPlusKMinus_ms1p5':[38,187],
    'SToK0K0_ms1p5':[38,187],
    'SToPi0Pi0_ms0p4':[10,50],
    'SToPi0Pi0_ms1p0':[25,125],
    'SToPiPlusPiMinus_ms0p4':[10,50],
    'SToPiPlusPiMinus_ms1p0':[25,125],
    'STodd_ms7':[100, 1000, 10000, 100000],
    'STodd_ms15':[100, 1000, 10000, 100000],
    'STodd_ms40':[100, 1000, 10000, 100000],
    'STodd_ms55':[100, 1000, 10000, 100000],
    'SToTauTau_ms7':[100, 1000, 10000, 100000],
    'SToTauTau_ms15':[100, 1000, 10000, 100000],
    'SToTauTau_ms40':[100, 1000, 10000, 100000],
    'SToTauTau_ms55':[100, 1000, 10000, 100000],
    'SToBB_ms15':[100, 1000, 10000, 100000],
    'SToBB_ms40':[100, 1000, 10000, 100000],
    'SToBB_ms55':[100, 1000, 10000, 100000],
    
#     'SToEE_ms4p0':[100,500],
#     'SToGammaGamma_ms4p0':[100,500],
#     'SToPiPlusPiMinus_ms4p0':[100,500],
    
}


prod = ['ggH', 'VBFH','ZH', 'WH', 'ttH','ggZH']
# prod=['ggH']

ntupler_version = 'V1p17/'
mc_path = {}
analyzer_version = 'v1/v163/'
mc_central_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_all/'+analyzer_version+'/normalized/'
analyzer_version = 'v2/v163/'
mc_private_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/'+ntupler_version+'/MC_Fall18/'+analyzer_version+'/normalized/'


for p in prod:
#     if 'WH' in p:continue
    for k in old_ctau.keys():
        for ct in old_ctau[k]:

            key = 'MC_'+p+'_'+k + '_'+str(ct)
            if (p == 'ggH' or p == 'VBFH') and ('ms7' in k or 'ms15' in k or 'ms40' in k or 'ms55' in k) and not (p =='VBFH' and 'dd' in k):
                mass = k[k.find('ms')+2:]
                if 'dd' in k and p == 'ggH':fpath[key] = mc_central_path+p+'_HToSSTodddd_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
                elif 'BB' in k and p =='ggH':fpath[key] = mc_central_path+p+'_HToSSTobbbb_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
                elif 'BB' in k and p =='VBFH':fpath[key] = mc_central_path+p+'_HToSSTo4b_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
                else:fpath[key] = mc_central_path+p+'_HToSSTo4Tau_MH-125_MS-'+mass+'_ctau-'+str(ct)+'_137000pb_weighted.root'
            else:
                
                if 'ggH' in p or 'ttH' in p: fpath[key] = mc_private_path+p+'_HToSS_'+k+'_pl'+str(ct)+'_137000pb_weighted.root'
                elif p == 'VBFH' or p=='ZH' or p == 'WH': fpath[key] = mc_private_path+p+'ToSS_'+k+'_pl'+str(ct)+'_137000pb_weighted.root'
                elif 'ggZH' in p:fpath[key] = mc_private_path+'ZHToSS_'+k+'_pl'+str(ct)+'_137000pb_weighted.root'
            if not os.path.exists(fpath[key]):print(key, fpath[key])
            

            
# mass = [15, 40, 55]
# if not decay == 'bbbb': mass = [7, 15, 40, 55]

# OLD_CTAU = np.array([100, 1000, 10000, 100000])#in mm

# ntupler_version = 'V1p17/'

data_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/Data2018/v5/v163/normalized/'
data_path = '/storage/af/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/Data2018/v5/v179/normalized/'#correct run number for DT veto

fpath['data'] = data_path + 'Run2_displacedJetMuonNtupler_V1p17_Data2016_Data2017_Data2018-HighMET_goodLumi.root'
# mc_path = {}



# fpath['hnl']= '/storage/af/user/christiw/HNL_electronType_ms5p0_plVe2_1e-5/HeavyNeutralLepton_Tree.root'                        
NEvents = {}
# NEvents_genweight = {}
for k,v in fpath.items():
    root_dir = uproot.open(v) 
    if not root_dir: 
        print(k, "zombie")
        continue
    tree[k] = root_dir['MuonSystem']
#     NEvents[k] = root_dir['NEvents'][1]
    NEvents[k] = root_dir['NEvents']._fEntries
    print(k)
#     if not 'data' in k: 
#         print(k, root_dir['NEvents']._fEntries)


root_dir = uproot.open('/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/HiggsPtWeights/ZHToggZH_HiggsPtReweight.root') 
h_reweight = root_dir['higgsPthiggsEta']

MC_ggH_STodd_ms3p0_100
MC_ggH_STodd_ms3p0_500
MC_ggH_SToEE_ms0p4_10
MC_ggH_SToEE_ms0p4_50
MC_ggH_SToGammaGamma_ms0p4_10
MC_ggH_SToGammaGamma_ms0p4_50
MC_ggH_SToKPlusKMinus_ms1p5_38
MC_ggH_SToKPlusKMinus_ms1p5_187
MC_ggH_SToK0K0_ms1p5_38
MC_ggH_SToK0K0_ms1p5_187
MC_ggH_SToPi0Pi0_ms0p4_10
MC_ggH_SToPi0Pi0_ms0p4_50
MC_ggH_SToPi0Pi0_ms1p0_25
MC_ggH_SToPi0Pi0_ms1p0_125
MC_ggH_SToPiPlusPiMinus_ms0p4_10
MC_ggH_SToPiPlusPiMinus_ms0p4_50
MC_ggH_SToPiPlusPiMinus_ms1p0_25
MC_ggH_SToPiPlusPiMinus_ms1p0_125
MC_ggH_STodd_ms7_100
MC_ggH_STodd_ms7_1000
MC_ggH_STodd_ms7_10000
MC_ggH_STodd_ms7_100000
MC_ggH_STodd_ms15_100
MC_ggH_STodd_ms15_1000
MC_ggH_STodd_ms15_10000
MC_ggH_STodd_ms15_100000
MC_ggH_STodd_ms40_100
MC_ggH_STodd_ms40_1000
MC_ggH_STodd_ms40_10000
MC_ggH_STodd_ms40_100000
MC_ggH_STodd_ms55_100
MC_ggH_STodd_ms55_1000
MC_ggH_STodd_ms55_10000
MC_ggH_STodd_ms55_100000
MC_ggH_SToTauTau_ms7_100
MC_ggH_SToTauTau_ms7_1000
MC_ggH_SToTauTau_ms7_10000
MC_ggH_SToTauTau_ms7_100000
MC_ggH_SToTauTau_ms15_

# Selections

In [15]:

jetPt_cut = 50

weight = {}
nhits1 = {}
nhits2 = {}
sel_ev = {}
cond = {}
cond_dtnoise_2016 = {}
ggZH_weight = {}
higgsEta = {}
higgsPt = {}

nCosmic = {}
deltaEta = {}
deltaRCluster = {}
jetPt = {}
jetTightPassId = {}
runNum = {}
evtNum = {}
lumiNum = {}
sel_ev_dtnoise_2016 = {}
nhits1_dtnoise_2016 = {}
nhits2_dtnoise_2016 = {}
cluster_index = ''
addNoiseFlag = 1
# 0: 2 CSC; 1: 2DT; 2: csc+dt
category = 2


for k in list(tree.keys()):
#     if not 'ggH' in k:continue
#     if  not 'data' in k:continue
########### SELECTION: CLUSTERS ############
    if 'data' in k: T = tree['data']
    else: T = tree[k]

    sel_csccluster = T.array('cscRechitCluster' + cluster_index + 'TimeSpreadWeightedAll')<20
    sel_csccluster = np.logical_and(sel_csccluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'MetEENoise_dPhi'))<1.2)
    sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'JetVetoPt')<30)
    sel_csccluster = np.logical_and(sel_csccluster, T.array('cscRechitCluster' + cluster_index + 'Me11Ratio')<1)
    sel_csccluster = np.logical_and(sel_csccluster, np.logical_not(np.logical_and(T.array('cscRechitClusterMuonVetoPt') >= 30, T.array('cscRechitClusterMuonVetoGlobal'))))
#     sel_csccluster = np.logical_and(sel_csccluster, np.abs(T.array('cscRechitCluster' + cluster_index + 'Eta'))<2.3)


    if 'data' in k: 
        sel_csccluster = np.logical_and(sel_csccluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')< 12.5, \
                                                                         T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') > -5))
    else: 
        sel_csccluster = np.logical_and(sel_csccluster, np.logical_and(T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')+0.66 < 12.5, \
                                                                         T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')+0.66 > -5))
        
    sel_dtcluster = np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1
    sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))
    sel_dtcluster = np.logical_and(sel_dtcluster, np.abs(T.array('dtRechitClusterJetVetoPt')) < 50)
    sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(T.array('dtRechitClusterMaxStation')==1, T.array('dtRechitClusterMaxStationRatio')>0.9)))

    
    cut = 5
    station = (T.array('dtRechitClusterNSegmentStation1')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>cut).astype(int)\
+(T.array('dtRechitClusterNSegmentStation3')>cut).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>cut).astype(int)

    max_station = np.maximum(np.maximum(np.maximum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))
    min_station = np.minimum(np.minimum(np.minimum(T.array('dtRechitClusterNSegmentStation1'), T.array('dtRechitClusterNSegmentStation2')), T.array('dtRechitClusterNSegmentStation3')), T.array('dtRechitClusterNSegmentStation4'))

    sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_or(station<4, min_station/max_station<0.4)) #remove if both clusters are 4 stations
    if addNoiseFlag and not 'data' in k: 
        sel_dtcluster = np.logical_and(sel_dtcluster, (T.array('dtRechitClusterSize')+T.array('dtRechitClusterNoiseHit')) >= 50)
    else: sel_dtcluster = np.logical_and(sel_dtcluster, T.array('dtRechitClusterSize') >= 50)



    if k in 'data': 
        sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(T.array('dtRechitClusterNoiseVeto')))
        sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(T.array('dtRechitClusterOverlap')))
    ###################### cosmic muon veto #############
    sel_cosmic = np.logical_and(T.array('dtRechitClusterNOppositeSegStation1')>0, T.array('dtRechitClusterNOppositeSegStation2')>0)
    sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation3')>0)
    sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation4')>0)
    sel_cosmic = np.logical_and(sel_cosmic, T.array('dtRechitClusterNOppositeSegStation1')+T.array('dtRechitClusterNOppositeSegStation2')+\
                               T.array('dtRechitClusterNOppositeSegStation3')+T.array('dtRechitClusterNOppositeSegStation4')>=6)
    nstation = (T.array('dtRechitClusterNSegmentStation1')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation2')>1).astype(int)\
    +(T.array('dtRechitClusterNSegmentStation3')>1).astype(int)+(T.array('dtRechitClusterNSegmentStation4')>1).astype(int)
    
    sel_dtcluster = np.logical_and(sel_dtcluster, np.logical_not(np.logical_and(nstation>=3, sel_cosmic)))

    noise_2016 = np.logical_and(T.array('dtRechitClusterMaxStation') == 2, T.array('dtRechitClusterWheel') == 1)
    noise_2016 = np.logical_and(noise_2016, T.array('dtRechitClusterPhi') > math.pi/12)
    noise_2016 = np.logical_and(noise_2016, T.array('dtRechitClusterPhi') < math.pi/4)
    sel_dtcluster_noise_2016 = np.logical_and(sel_dtcluster, np.logical_not(noise_2016))


########### SELECTION: JETS ############
    
    sel_jet = np.logical_and(T.array('jetPt') > 30, np.abs(T.array('jetEta')) < 2.4 )
    sel_jet = np.logical_and(T.array('jetTightPassId'), sel_jet)


            
########### SELECTION: NOISE IN DT ############
    
    spike = np.logical_and( T.array('nDTRechitsSector')[:,0,0,7]>50,  T.array('nDTRechitsSector')[:,0,0,7]+T.array('nDTRechitsSector')[:,0,0,8]+T.array('nDTRechitsSector')[:,0,0,9]>120)
    spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,8]>25)
    spike = np.logical_and(spike, T.array('nDTRechitsSector')[:,0,0,9]>10)

    
    
########### SELECTION: EVENTS ############

    sel_ev[k] = T.array('METNoMuTrigger')
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise') >= 200)
    sel_ev[k] = np.logical_and(sel_ev[k] , sel_jet.sum()>=1)
    sel_ev[k] = np.logical_and(sel_ev[k], (T.array('nDtRings')+T.array('nCscRings'))<10)
    sel_ev[k] = np.logical_and(sel_ev[k],T.array('Flag2_all'))
    sel_ev[k] = np.logical_and(sel_ev[k] , np.logical_not(spike))
########### BRANCHES ############
                            
    if category == 0:
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 2)
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 0)
        cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
                        T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,1], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,1])<2
        
       
        nhits1[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,0]
        nhits2[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,1]
#         deltaRCluster[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
#                         T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,1], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,1])
    elif category == 1:
        
        
        # data
        if k == 'data':
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 0)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 2)
            nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,0]
            nhits2[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,1]
        else:
            # reweight to 1.5/fb, for events with dt noise
            sel_ev_dtnoise_2016[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 0)
            sel_ev_dtnoise_2016[k]  = np.logical_and(sel_ev_dtnoise_2016[k],sel_dtcluster_noise_2016.sum() == 2)

            if addNoiseFlag and not 'data' in k:
                nhits1_dtnoise_2016[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,0]
                nhits2_dtnoise_2016[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,1]
            else:
                nhits1_dtnoise_2016[k] =  T.array('dtRechitClusterSize')[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,0]
                nhits2_dtnoise_2016[k] =  T.array('dtRechitClusterSize')[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,1]

            # normal condition
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 2)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 0)

            if addNoiseFlag and not 'data' in k:
                nhits1[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][:,0]
                nhits2[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][:,1]
            else:
                nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,0]
                nhits2[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][:,1]
        
            nhits1[k] = np.hstack((nhits1[k], nhits1_dtnoise_2016[k]))
            nhits2[k] = np.hstack((nhits2[k], nhits2_dtnoise_2016[k]))

    elif category == 2:
        # data
        if k == 'data':
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum()== 1)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum()== 1)
            cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
                            T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0], T.array('dtRechitCluster' + cluster_index + 'Phi')[sel_dtcluster][sel_ev[k]][:,0])<2.5
            nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
            nhits2[k] =  T.array('cscRechitClusterSize')[sel_csccluster][sel_ev[k]][cond[k]][:,0]
        else:
            sel_ev_dtnoise_2016[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 1)
            sel_ev_dtnoise_2016[k]  = np.logical_and(sel_ev_dtnoise_2016[k],sel_dtcluster_noise_2016.sum() == 1)

            sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 1)
            sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum() == 1)


            cond[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev[k]][:,0],\
                            T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster][sel_ev[k]][:,0], T.array('dtRechitCluster' + cluster_index + 'Phi')[sel_dtcluster][sel_ev[k]][:,0])<2.5
            if addNoiseFlag and not 'data' in k:
                nhits1[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
            else:
                nhits1[k] =  T.array('dtRechitClusterSize')[sel_dtcluster][sel_ev[k]][cond[k]][:,0]
            nhits2[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev[k]][cond[k]][:,0]

            # reweight to 1.5/fb, for events with dt noise
            cond_dtnoise_2016[k] = deltaR(T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_csccluster][sel_ev_dtnoise_2016[k]][:,0], T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_csccluster][sel_ev_dtnoise_2016[k]][:,0],\
                            T.array('dtRechitCluster' + cluster_index + 'Eta')[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,0], T.array('dtRechitCluster' + cluster_index + 'Phi')[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][:,0])<2.5
            if addNoiseFlag and not 'data' in k:
                nhits1_dtnoise_2016[k] =  (T.array('dtRechitClusterNoiseHit')+T.array('dtRechitClusterSize'))[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]][:,0]
            else:
                nhits1_dtnoise_2016[k] =  T.array('dtRechitClusterSize')[sel_dtcluster_noise_2016][sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]][:,0]
            nhits2_dtnoise_2016[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_csccluster][sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]][:,0]
            nhits1[k] = np.hstack((nhits1[k], nhits1_dtnoise_2016[k]))
            nhits2[k] = np.hstack((nhits2[k], nhits2_dtnoise_2016[k]))
    else:
        assert(False)
    if not k in 'data':
        if category == 0:
            higgsPt[k] = T.array('gHiggsPt')[sel_ev[k]][cond[k]]
            higgsEta[k] = T.array('gHiggsEta')[sel_ev[k]][cond[k]]
            ggZH_weight[k]=h_reweight.values[np.argmax(h_reweight.edges[0]>higgsPt[k][:,None],axis=1)-1, np.argmax(h_reweight.edges[1]>np.abs(higgsEta[k])[:,None],axis=1)-1]
            if 'ggH' in k: weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))[sel_ev[k]][cond[k]]
            else:weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev[k]][cond[k]]
            if 'ggZH' in k: weight[k] *= ggZH_weight[k]
        elif category == 1:
            higgsPt[k] = T.array('gHiggsPt')[sel_ev[k]]
            higgsEta[k] = T.array('gHiggsEta')[sel_ev[k]]
            ggZH_weight[k]=h_reweight.values[np.argmax(h_reweight.edges[0]>higgsPt[k][:,None],axis=1)-1, np.argmax(h_reweight.edges[1]>np.abs(higgsEta[k])[:,None],axis=1)-1]
            if 'ggH' in k: weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))[sel_ev[k]]
            else:weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev[k]]
            if 'ggZH' in k: weight[k] *= ggZH_weight[k]
            weight[k] *= (137-1.5)/137

            higgsPt[k] = T.array('gHiggsPt')[sel_ev_dtnoise_2016[k]]
            higgsEta[k] = T.array('gHiggsEta')[sel_ev_dtnoise_2016[k]]
            ggZH_weight[k]=h_reweight.values[np.argmax(h_reweight.edges[0]>higgsPt[k][:,None],axis=1)-1, np.argmax(h_reweight.edges[1]>np.abs(higgsEta[k])[:,None],axis=1)-1]
            if 'ggH' in k: weight_temp = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))[sel_ev_dtnoise_2016[k]]
            else:weight_temp = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev_dtnoise_2016[k]]
            if 'ggZH' in k: weight_temp *= ggZH_weight[k]
            weight[k] = np.hstack((weight[k], weight_temp * 1.5/137))
        else:
            higgsPt[k] = T.array('gHiggsPt')[sel_ev[k]][cond[k]]
            higgsEta[k] = T.array('gHiggsEta')[sel_ev[k]][cond[k]]
            ggZH_weight[k]=h_reweight.values[np.argmax(h_reweight.edges[0]>higgsPt[k][:,None],axis=1)-1, np.argmax(h_reweight.edges[1]>np.abs(higgsEta[k])[:,None],axis=1)-1]
            if 'ggH' in k: weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))[sel_ev[k]][cond[k]]
            else:weight[k] = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev[k]][cond[k]]
            if 'ggZH' in k: weight[k] *= ggZH_weight[k]
            weight[k] *= (137-1.5)/137

            higgsPt[k] = T.array('gHiggsPt')[sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]]
            higgsEta[k] = T.array('gHiggsEta')[sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]]
            ggZH_weight[k]=h_reweight.values[np.argmax(h_reweight.edges[0]>higgsPt[k][:,None],axis=1)-1, np.argmax(h_reweight.edges[1]>np.abs(higgsEta[k])[:,None],axis=1)-1]
            if 'ggH' in k: weight_temp = (T.array('weight')*T.array('pileupWeight')*T.array('higgsPtWeight')*T.array('metSF'))[sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]]
            else:weight_temp = (T.array('weight')*T.array('pileupWeight')*T.array('metSF'))[sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]]
            if 'ggZH' in k: weight_temp *= ggZH_weight[k]
            weight[k] = np.hstack((weight[k], weight_temp * 1.5/137))
            

In [16]:
# datacard_version  = 'v1' # ABCD for all categories
datacard_version  = 'v2' # ABCD for CSC-DT and 3 bins for CSC-CSC and DT-DT
datacard_version  = 'v3' # signal uncertainties and corrections applied
datacard_version  = 'v4' # global muon veto for csc and loose id muon veto for dt
datacard_version  = 'v5' # global muon veto + eta cut for csc and loose id muon veto for dt
datacard_version  = 'v6' # global muon veto for csc and loose id muon veto + station veto +tight ID jet veto for dt

datacard_version  = 'v7' # deltaEta cut 2.5 (DT-CSC) and 1.5 (CSC-CSC)
datacard_version  = 'v8' # deltaEta cut 2 (DT-CSC) and 1.5 (CSC-CSC)

datacard_version  = 'v9' # deltaR cut for CSC and DT
datacard_version  = 'v10' # change ABCD rateParam to the simple case
datacard_version  = 'v11' # add dtnoise2016 issue
# datacard_version  = 'v12' # eta<2.1

# datacard_version  = 'v13' # ETA<2.2
# datacard_version  = 'v14' # ETA<2.0

# datacard_version  = 'v14' # ETA<2.2

datacard_version  = 'v15' # change names for combination, same number of events as v11




In [17]:
if category == 0:prefix = 'csccsc_'
elif category == 1:prefix = 'dtdt_'
else: prefix = 'dtcsc_'

# signal uncertainties

In [18]:
cscCluster = {}
time_spread = {}
dtCluster = {}
higgsPtWeight = {}
JES = {}
pileup = {}
# ctau_reweight = {}
mc_stats = {}
scaling = {}
lumi = {}
time_spread = {}
time = {}
theory = {}
pdf = {}
readout = {}
ggH_higgsPt , VBFH_higgsPt , WH_higgsPt , ZH_higgsPt , ttH_higgsPt , ggZH_higgsPt =({} for i in range(6)) 
ggH_xsec , VBFH_xsec , WH_xsec , ZH_xsec , ttH_xsec , ggZH_xsec =({} for i in range(6)) 
ggH_pdf , VBFH_pdf , WH_pdf , ZH_pdf , ttH_pdf , ggZH_pdf =({} for i in range(6)) 
Nrechit_weight_file = '/storage/user/christiw/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/Uncertainties/Nrechit_weight.txt'

cut_based_unc = [0.0921, 0.0852] # muon Eta reweighted
clustering_unc = [0.1506, 0.1047]
if category == 2 or datacard_version == 'v1': bins = 4
else: bins = 3
print('category: '+ str(category))
print('bins: '+ str(bins))

            
for m in old_ctau.keys():
    for ct in old_ctau[m]:
        for p in prod:
            if p == 'others':k = 'MC_ggH_'+str(m)+'_'+str(ct)   
            else:k = 'MC_'+p+'_'+str(m)+'_'+str(ct)         
            if p == 'others': key = 'MC_others_'+str(m)+'_'+str(ct)   
            else: key = k
            if p == 'others':k = 'MC_ggH_'+str(m)+'_'+str(ct)   
            else:k = 'MC_'+p+'_'+str(m)+'_'+str(ct)         
            if p == 'others': key = 'MC_others_'+str(m)+'_'+str(ct)   
            else: key = k
            
            # CSC hits
            if category == 0:
#                 cscCluster_mc[key] = [0.02] * bins
#                 cscCluster_data[key] = [0.012] * bins
                cscCluster[key] = [0.02] * bins
#                 time_spread_mc[key] = [0.037*2] * bins
#                 time_spread_data[key] = [0.039*2] * bins
                time_spread[key] = [0.1075] * bins
                dtCluster[key] = [0.0] * bins
                JES[key] = [0.0351,0.0573, 0.0470] 
                pileup[key] = [0.0102,0.0144,0.0183]

            elif category == 1:
                cscCluster[key] = [0.0] * bins
                time_spread[key] = [0.0] * bins
                dtCluster[key] = [0.032] * bins
                JES[key] = [0.027, 0.036,0.079]
                pileup[key] = [0.013, 0.0152, 0.0783]
            else: 
#                 cscCluster_mc[key] = [0.013] * bins
#                 cscCluster_data[key] = [0.017] * bins
#                 time_spread_mc[key] = [0.037] * bins
#                 time_spread_data[key] = [0.039] * bins
                
                cscCluster[key] = [0.013] * bins
                time_spread[key] = [0.054] * bins
                dtCluster[key] = [0.011] * bins
                JES[key] = [0.0332, 0.0306, 0.1211,0.039]
                pileup[key] = [0.011,0.019, 0.018,0.025]


            lumi[key] = [0.016]*bins
            


            ggH_higgsPt[key], VBFH_higgsPt[key], WH_higgsPt[key] , ZH_higgsPt[key] , ttH_higgsPt[key] , ggZH_higgsPt[key] = ([0.0]*bins*2,)*6
            ggH_xsec[key], VBFH_xsec[key], WH_xsec[key] , ZH_xsec[key] , ttH_xsec[key] , ggZH_xsec[key] = ([0.0]*bins*2,)*6
            ggH_pdf[key] , VBFH_pdf[key] , WH_pdf[key] , ZH_pdf[key] , ttH_pdf[key] , ggZH_pdf[key] = ([0.0]*bins,)*6
            if 'ggH' in key: 
#                 if category == 2 or datacard_version == 'v1': ggH_higgsPt[key] = [0.205, 0.205,0.207,0.208, 0.133,0.133,0.134,0.134] #down*4/up*4
#                 else:  ggH_higgsPt[key] = [0.205, 0.205,0.207, 0.133,0.133,0.134] #down*4/up*4
                    
                if category == 0:ggH_higgsPt[key] = [0.2066,0.2069, 0.2069,0.1339,0.134,0.1341] #down*3/up*3
                elif category == 1:ggH_higgsPt[key] = [0.2074,0.2077, 0.2061,0.1342,0.1343,0.1336] #down*3/up*3
                else:ggH_higgsPt[key] = [0.2071, 0.2066, 0.2084, 0.2076, 0.134,0.134,0.1345,0.1343] #down*4/up*4
                ggH_xsec[key] = [0.067]*bins+[0.046]*bins #down/up
                ggH_pdf[key] = [0.032]*bins

            elif 'VBFH' in key: 
#                 if category == 2 or datacard_version == 'v1':VBFH_higgsPt[key] = [0.006, 0.006, 0.006, 0.006, 0.010, 0.010, 0.010, 0.010] #down/up
#                 else: VBFH_higgsPt[key] = [0.006, 0.006, 0.006, 0.010, 0.010, 0.010] #down/up
                if category == 0:VBFH_higgsPt[key] = [0.0364,0.0484,0.0811,0.0084,0.0241,0.0161] #down*3/up*3
                elif category == 1:VBFH_higgsPt[key] = [0.0154, 0.0332,0.0823,0.0102,0.024,0.0485]
                else:VBFH_higgsPt[key] = [0.0152,0.0306,0.1045,0.0208, 0.0096,0.0071, 0.0351,0.0109]
                VBFH_xsec[key] = [0.003]*bins+[0.004]*bins #down/up
                VBFH_pdf[key] = [0.021]*bins
            elif 'ggZH' in key:
                if category == 0:ggZH_higgsPt[key] = [0.114,0.0501,0.2873,0.0199,0.0134,0.1375]
                elif category == 1:ggZH_higgsPt[key] = [0.0426,0.2433,0.2063,0.0229,0.0396,0.0075]
                else:ggZH_higgsPt[key] = [0.0995, 0.0998, 0.8319,0.1495,0.0194,0.0144,0.2761,0.0753]
                ggZH_xsec[key] = [0.251]*bins+[0.189]*bins #down/up
                ggZH_pdf[key] = [0.024]*bins
            elif 'ZH' in key: 
                if category == 0:ZH_higgsPt[key] = [0.0658, 0.055, 0.3026, 0.0135, 0.013, 0.1221]
                elif category == 1:ZH_higgsPt[key] = [0.0674, 0.0771, 0.03348, 0.0265, 0.022, 0.0912]
                else:ZH_higgsPt[key] = [0.1875, 0.0629, 0.2784, 0.731, 0.0049, 0.0107, 0.0934, 0.0251]
                ZH_xsec[key] = [0.006]*bins+[0.005]*bins #down/up
                ZH_pdf[key] = [0.019]*bins
            elif 'WH' in key: 
                if category == 0:WH_higgsPt[key] = [0.099, 0.0525, 0.3833, 0.0162, 0.0132, 0.0854]
                elif category == 1:WH_higgsPt[key] = [0.0241, 0.1452, 0.1078, 0.0099, 0.0457, 0.0478]
                else:WH_higgsPt[key] = [0.0772, 0.1123, 0.5623, 0.204, 0.0102, 0.0313, 0.1362, 0.0541]
                WH_xsec[key] = [0.007]*bins+[0.005]*bins #down/up
                WH_pdf[key] = [0.019]*bins
            elif 'ttH' in key: 
                if category == 0:ttH_higgsPt[key] = [0.0919, 0.2877, 0.3674, 0.0245, 0.0976, 0.1691]
                elif category == 1:ttH_higgsPt[key] = [0.0144,0.0076, 0.4021, 0.0024, 0.048, 0.166]
                else:ttH_higgsPt[key] = [0.0092, 0.2868, 0.2367, 0.0844, 0.0107, 0.0635, 0.1291, 0.0287]
                ttH_xsec[key] = [0.092]*bins+[0.058]*bins #down/up
                ttH_pdf[key] = [0.036]*bins
            mc_stats[k] = []
            if p == 'ggZH': scaling[key] = [0.2]*bins
            else: scaling[key] = [0] * bins


# sig_eff = [lumi, cscCluster_mc, cscCluster_data, time_spread_mc, time_spread_data, dtCluster, JES,pileup, \
#            ggH_higgsPt, VBFH_higgsPt, WH_higgsPt, ZH_higgsPt, ttH_higgsPt, ggZH_higgsPt,  \
#            ggH_xsec, VBFH_xsec, WH_xsec, ZH_xsec, ttH_xsec, ggZH_xsec, \
#            ggH_pdf,VBFH_pdf, WH_pdf, ZH_pdf, ttH_pdf, ggZH_pdf,\
#            scaling, mc_stats]
# sig_eff_name = ['lumi','cscCluster_nhits_mc', 'cscCluster_nhits_data', 'csc_timeSpread_mc', 'csc_timeSpread_data', 'dtCluster_nhits', 'JES','pileup', \
#                 'ggH_higgsPt', 'VBFH_higgsPt', 'WH_higgsPt', 'ZH_higgsPt', 'ttH_higgsPt', 'ggZH_higgsPt',\
#                 'ggH_xsec', 'VBFH_xsec', 'WH_xsec', 'ZH_xsec', 'ttH_xsec', 'ggZH_xsec', \
#                 'ggH_pdf','VBFH_pdf', 'WH_pdf', 'ZH_pdf', 'ttH_pdf', 'ggZH_pdf',\
#                 'ggZH_reweight','mc_stats']
sig_eff = [lumi, cscCluster, time_spread, dtCluster, JES,pileup, \
           ggH_higgsPt, VBFH_higgsPt, WH_higgsPt, ZH_higgsPt, ttH_higgsPt, ggZH_higgsPt,  \
           ggH_xsec, VBFH_xsec, WH_xsec, ZH_xsec, ttH_xsec, ggZH_xsec, \
           ggH_pdf,VBFH_pdf, WH_pdf, ZH_pdf, ttH_pdf, ggZH_pdf,\
           scaling, mc_stats]
sig_eff_name = ['lumi','csc_clusterSyst',  'csc_time_spread', 'dt_clusterSyst', 'JES','pileup', \
                'ggH_higgsPt', 'VBFH_higgsPt', 'WH_higgsPt', 'ZH_higgsPt', 'ttH_higgsPt', 'ggZH_higgsPt',\
                'ggH_xsec', 'VBFH_xsec', 'WH_xsec', 'ZH_xsec', 'ttH_xsec', 'ggZH_xsec', \
                'ggH_pdf','VBFH_pdf', 'WH_pdf', 'ZH_pdf', 'ttH_pdf', 'ggZH_pdf',\
                'ggZH_reweight',prefix+'mc_stats']



# csc: jet, muon, time, time spread, clustereff_unc, readout
# theory uncertainty for now
assert(len(sig_eff)==len(sig_eff_name))

category: 2
bins: 4


In [19]:
ctaus = []
for i in range(1,8):
    if i == 1:ctaus+=(list(np.arange(1,2*10**i,i)))
    elif i == 2:ctaus+=list(np.arange(2*10**(i-1),10**i,10.0**i/20))
    else: ctaus+=list(np.arange(10**(i-1),10**i,10.0**i/40))
# for i in range(1,8):
#     if i == 1:ctaus+=(list(np.arange(1,2*10**i,i)))
#     elif i == 2:ctaus+=list(np.arange(2*10**(i-1),10**i,10.0**i/10))
#     else: ctaus+=list(np.arange(10**(i-1),10**i,10.0**i/10))

ctaus = [str(int(item)) for item in ctaus]    
print(ctaus)
# ctaus = ['300']
# ctaus = ['20', '25', '30', '35', '40', '45', '50', '55', '60', '65', '70', '75', '80', '85', '90', '95', '100']
# ctaus = ['1.25', '1.5', '1.75', '2.25','2.5','2.75','3.25','3.5','3.75',]

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '25', '30', '35', '40', '45', '50', '55', '60', '65', '70', '75', '80', '85', '90', '95', '100', '125', '150', '175', '200', '225', '250', '275', '300', '325', '350', '375', '400', '425', '450', '475', '500', '525', '550', '575', '600', '625', '650', '675', '700', '725', '750', '775', '800', '825', '850', '875', '900', '925', '950', '975', '1000', '1250', '1500', '1750', '2000', '2250', '2500', '2750', '3000', '3250', '3500', '3750', '4000', '4250', '4500', '4750', '5000', '5250', '5500', '5750', '6000', '6250', '6500', '6750', '7000', '7250', '7500', '7750', '8000', '8250', '8500', '8750', '9000', '9250', '9500', '9750', '10000', '12500', '15000', '17500', '20000', '22500', '25000', '27500', '30000', '32500', '35000', '37500', '40000', '42500', '45000', '47500', '50000', '52500', '55000', '57500', '60000', '62500', '65000', '67500', '70000', '72500', '75000', '77500', '80000

# make datacard (ABCD)

In [20]:
%%time

import os
import math
import importlib
importlib.reload(sys.modules['helper'])
from helper import  make_datacard_2tag, make_datacard_2tag_3bins, weight_calc
N_RECHIT_CUTS = np.arange(60,220, 10)

# ctaus = []
# for i in range(1,8):
#     if i == 1:ctaus+=(list(np.arange(1,2*10**i,i)))
#     elif i == 2:ctaus+=list(np.arange(2*10**(i-1),10**i,10.0**i/10))
#     else: ctaus+=list(np.arange(10**(i-1),10**i,10.0**i/40))
# ctaus = [str(int(item)) for item in ctaus]    
# ctaus = ['20', '25', '30', '35', '40', '45', '50', '55', '60', '65', '70', '75', '80', '85', '90', '95', '100']
# ctaus = ['1.25', '1.5', '1.75', '2.25','2.5','2.75','3.25','3.5','3.75',]
# ctaus = ['0.1','0.125', '0.15', '0.175', '0.2','0.25','0.3','0.35','0.4','0.5','0.6','0.7','0.8','0.9']
print(ctaus)


UNBLIND = 1
# 0 use exponential fit, 1 unblind ABC, 2 unblind ABCD
N_RECHIT_CSC = 100
N_RECHIT_DT = 80


#####
# a = c*c1*c2
# b = c1* c
# c = c
# d = c2*c
# start adding signal
#####
outDataCardsDir = '/storage/af/user/christiw/login-1/christiw/LLP/CMSSW_10_2_13/src/HiggsAnalysis/MuonSystemLimit/combine/datacards_2tag/'+\
ntupler_version+analyzer_version+'/'
outDataCardsDir += datacard_version +'/'
if UNBLIND == 0: outDataCardsDir += 'blind/'
elif UNBLIND == 1: outDataCardsDir += 'unblindABC/'
else:outDataCardsDir += 'unblind/'
outDataCardsDir += 'category'+str(category)+'/'
if not os.path.isdir(outDataCardsDir):os.makedirs(outDataCardsDir)
print(outDataCardsDir)
if category == 2 or datacard_version == 'v1': bins = 4
else: bins = 3


#############
# BACKGROUND
#############

bkg_unc = [] #ABCD closure, sum of stats from two VR
bkg_unc_name = []
if UNBLIND == 0:
    ##### v138 #####
    if 'v138' in analyzer_version:
        if category == 0:bkg_rate = np.array([0.062*0.073,0.073,1,0.062])*59 #2csc
        elif category == 1:bkg_rate = np.array([0.02*0.01,0.02,1,0.01])*44 # dt
        elif category == 2:bkg_rate = np.array([0.07*0.15,0.07,1,0.15])*66 #csc+dt
    elif 'v141' in analyzer_version:
        if datacard_version == 'v1':
            if category == 0:bkg_rate = np.array([0.03*0.03,0.03,1 ,0.03])*18 #2csc
            elif category == 1:bkg_rate = np.array([0.03*0.03,0.03,1, 0.03])*19 # dt
            elif category == 2:bkg_rate = np.array([0.009*0.025,0.009,1,0.025])*35 #csc+dt
        elif datacard_version == 'v2':
            if category == 0:bkg_rate = np.array([0.03*0.03,0.03*2,1])*18 #2csc
            elif category == 1:bkg_rate = np.array([0.03*0.03,0.03*2,1])*19 # dt
            elif category == 2:bkg_rate = np.array([0.009*0.025,0.009,1,0.025])*35 #csc+dt
    elif 'v146' in analyzer_version: #with dtcosmic muon veto
        if datacard_version == 'v2':
#             if category == 0:bkg_rate = np.array([0.02, 1.2, 18])
#             elif category == 1:bkg_rate = np.array([0.02, 1.2, 17.8])
#             elif category == 2:bkg_rate = np.array([0.01, 0.3, 34.2, 0.9])
            if category == 0:bkg_rate = np.array([0.09, 2.5, 18])
            elif category == 1:bkg_rate = np.array([0.02, 1.2, 17.9])
            elif category == 2:bkg_rate = np.array([0.02, 0.7, 34.2, 0.9])
    observation= bkg_rate #blinded observation

elif UNBLIND == 1:
    k = 'data'
    if category <= 1:
        if category == 0: N_RECHIT_CUT = N_RECHIT_CSC
        else: N_RECHIT_CUT = N_RECHIT_DT
#             cond_a = np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT)#bin A
        b = np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT)#bin A
        b = np.count_nonzero(np.logical_or(b, np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT)))
        c = np.count_nonzero(np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT))
        bkg_rate = np.array([b**2/4.0/c, b, c])

    else:
        N_RECHIT_CUT_1 = N_RECHIT_DT
        N_RECHIT_CUT_2 = N_RECHIT_CSC
        cond_a = np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2)#bin A
        b = np.count_nonzero(np.logical_and(nhits1[k]<N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2))
        d = np.count_nonzero(np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]<N_RECHIT_CUT_2))
        c = np.count_nonzero(np.logical_and(nhits1[k]<N_RECHIT_CUT_1, nhits2[k]<N_RECHIT_CUT_2))
        bkg_rate = np.array([b*d/c, b, c, d])
    observation= bkg_rate #blinded observation

    print('category: '+str(category))
    print('background: ', bkg_rate)
else:#unblind
    k = 'data'
    if category <= 1:
        if category == 0: N_RECHIT_CUT = N_RECHIT_CSC
        else: N_RECHIT_CUT = N_RECHIT_DT
        a = np.count_nonzero(np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT))#bin A
        b = np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT)#bin A
        b = np.count_nonzero(np.logical_or(b, np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT)))
        c = np.count_nonzero(np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT))
        bkg_rate = np.array([b**2/4.0/c, b, c])
        observation = [a,b,c]
    else:
        N_RECHIT_CUT_1 = N_RECHIT_DT
        N_RECHIT_CUT_2 = N_RECHIT_CSC
        a = np.count_nonzero(np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2))#bin A
        b = np.count_nonzero(np.logical_and(nhits1[k]<N_RECHIT_CUT_1, nhits2[k]>=N_RECHIT_CUT_2))
        d = np.count_nonzero(np.logical_and(nhits1[k]>=N_RECHIT_CUT_1, nhits2[k]<N_RECHIT_CUT_2))
        c = np.count_nonzero(np.logical_and(nhits1[k]<N_RECHIT_CUT_1, nhits2[k]<N_RECHIT_CUT_2))
        bkg_rate = np.array([b*d/c, b, c, d])
        observation = [a,b,c, d]
print('category: '+str(category))
print('background: ', bkg_rate)
print('observation: ', observation)

    
#############
# SIGNAL
#############


sig_norm = []
# for m in mass:
#     for ct in ctaus:
        
for m in old_ctau.keys():
    for ct in ctaus:
        if len(prod) == 5: modelName = 'allProd_HToSS_'
        elif len(prod) ==6:  modelName = 'allProd_withggZH_HToSS_'
        else: modelName = ('_').join(prod)+'_HToSS_'
        modelName = modelName +str(m)+'_ctau'+str(ct.replace('.','p'))+'mm_nCsc'+str(N_RECHIT_CSC)+'_nDt'+str(N_RECHIT_DT)
        OLD_CTAU = old_ctau[m].copy()
                


        signal_rate = {}
        mc_stat_unc = {}
        gmn = {}
        sig_unc = {}
        
        if '.' in ct:ctf = float(ct)
        else: ctf = int(ct)
        ct_list = 10**int(math.log10(ctf))

        if ctf < OLD_CTAU[0]: ct_list = [OLD_CTAU[0]]
        elif ctf>OLD_CTAU[-1]: ct_list = [OLD_CTAU[-1]]
        elif ctf in OLD_CTAU: ct_list = [ctf]
        elif len(OLD_CTAU) == 2:ct_list = OLD_CTAU.copy()
        else:ct_list = [ct_list,ct_list*10]
        for p in prod:      
            signal_rate[p] = np.zeros((bins,))
            mc_stat_unc[p] = np.zeros((bins,))
            gmn[p] = np.zeros((bins,))
            sig_unc[p] = []
            for i, ct0 in enumerate(ct_list):
                k = 'MC_'+p+'_'+str(m)+'_'+str(ct0)

                T = tree[k]
                if np.count_nonzero(sel_ev[k])==0:continue
                if category == 0:
                    gLLP_ctau = T.array('gLLP_ctau')[sel_ev[k]][cond[k]]
                    weight_ctau = weight_calc(gLLP_ctau, ctf/10, int(ct0)/10) # convert everything to cm
                    gLLP_ctau = np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][cond[k]]
                    if len(ct_list) == 1:weight_cond = gLLP_ctau >= 0
                    else:
                        if i == 0 : weight_cond = gLLP_ctau<int(ct_list[0]/2)
                        else: weight_cond = gLLP_ctau>=int(ct_list[0]/2)
                    w = weight[k]*weight_ctau
                
                elif category == 1:
                    gLLP_ctau = np.vstack((T.array('gLLP_ctau')[sel_ev[k]], T.array('gLLP_ctau')[sel_ev_dtnoise_2016[k]]))
                    weight_ctau = weight_calc(gLLP_ctau, ctf/10, int(ct0)/10) # convert everything to cm
                    gLLP_ctau = np.hstack((np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]], np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev_dtnoise_2016[k]]))
                    
                else: 
                    gLLP_ctau = np.vstack((T.array('gLLP_ctau')[sel_ev[k]][cond[k]], T.array('gLLP_ctau')[sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]]))
                    weight_ctau = weight_calc(gLLP_ctau, ctf/10, int(ct0)/10) # convert everything to cm
                    gLLP_ctau = np.hstack((np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev[k]][cond[k]], np.sum(T.array('gLLP_ctau'), axis = 1)[sel_ev_dtnoise_2016[k]][cond_dtnoise_2016[k]]))
                
                if len(ct_list) == 1:weight_cond = gLLP_ctau >= 0
                else:
                    if i == 0 : weight_cond = gLLP_ctau<int(ct_list[0]/2)
                    else: weight_cond = gLLP_ctau>=int(ct_list[0]/2)
                w = weight[k]*weight_ctau
                    
                if category == 2:
                    a = np.logical_and(np.logical_and(nhits1[k]>=N_RECHIT_DT, nhits2[k]>=N_RECHIT_CSC), weight_cond)
                    
                    signal_rate[p][0]+=np.sum(w[a])
                    mc_stat_unc[p][0]+=np.sum(w[a]**2)
                    if len(w[a]) == 1:gmn[p][0] += len(w[a])
                    else: gmn[p][0] += len(w[a])/2
                    
                    b = np.logical_and(np.logical_and(nhits1[k]<N_RECHIT_DT, nhits2[k]>=N_RECHIT_CSC), weight_cond)
                    signal_rate[p][1]+=np.sum(w[b])
                    mc_stat_unc[p][1]+=np.sum(w[b]**2)
                    if len(w[b]) == 1:gmn[p][1] += len(w[b])
                    else: gmn[p][1] += len(w[b])/2
                    c = np.logical_and(np.logical_and(nhits1[k]<N_RECHIT_DT, nhits2[k]<N_RECHIT_CSC), weight_cond)
                    signal_rate[p][2]+=np.sum(w[c])
                    mc_stat_unc[p][2]+=np.sum(w[c]**2)
                    if len(w[c]) == 1:gmn[p][2] += len(w[c])
                    else: gmn[p][2] += len(w[c])/2
                    
                    d = np.logical_and(np.logical_and(nhits1[k]>=N_RECHIT_DT, nhits2[k]<N_RECHIT_CSC), weight_cond)
                    signal_rate[p][3]+=np.sum(w[d])
                    mc_stat_unc[p][3]+=np.sum(w[d]**2)
                    if len(w[d]) == 1:gmn[p][3] += len(w[d])
                    else: gmn[p][3] += len(w[d])/2
                else:
                    ####
                    # a is signal region, b has one cluster pass Nrechit cut, C has no cluster passing Nrechit cut
                    ####
                    if category == 0:N_RECHIT_CUT = N_RECHIT_CSC
                    else: N_RECHIT_CUT = N_RECHIT_DT
                    a = np.logical_and(np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT), weight_cond)
                    signal_rate[p][0]+=np.sum(w[a])
                    mc_stat_unc[p][0]+=np.sum(w[a]**2)
                    if category == 0 or len(w[a]) == 1:gmn[p][0] += len(w[a])
                    else: gmn[p][0] += len(w[a])/2
                    b = np.logical_and(np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT), weight_cond)
                    d = np.logical_and(np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT), weight_cond)
                    b = np.logical_or(b, d)
                    signal_rate[p][1]+=np.sum(w[b])
                    mc_stat_unc[p][1]+=np.sum(w[b]**2)
                    if category == 0 or len(w[b]) == 1:gmn[p][1] += len(w[b])
                    else: gmn[p][1] += len(w[b])/2
                        
                    c = np.logical_and(np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT), weight_cond)
                    signal_rate[p][2]+=np.sum(w[c])
                    mc_stat_unc[p][2]+=np.sum(w[c]**2)
                    if category == 0 or len(w[c]) == 1:gmn[p][2] += len(w[c])
                    else: gmn[p][2] += len(w[c])/2
                for j in range(bins):
                    if signal_rate[p][j]<0.0:
                        signal_rate[p][j]=0.0
                        mc_stat_unc[p][j]=0.0
                        gmn[p][j] = 0.0
                    if gmn[p][j]>400:gmn[p][j] = 0.0

                        
                        
            mc_stat_unc[p] = np.nan_to_num(signal_rate[p]/np.sqrt(mc_stat_unc[p]))**2 #gmn
            mc_stat_unc[p][mc_stat_unc[p] == np.inf] = 0.0

            mc_stat_unc[p] = list(mc_stat_unc[p])
            mc_stat_unc[p] = list(gmn[p])
            if p == 'others': k = k.replace("ggH", "others")
            for j, ele in enumerate(sig_eff):# go through each uncertainty
                if j == len(sig_eff)-1: # if mc_stats
                    sig_unc[p].append(mc_stat_unc[p])
                else:
                    sig_unc[p].append(ele[k])


        norm = np.sum(signal_rate['ggH'])/4
        #only used for calculation, the signal yield is not changed in datacard

#         print(signal_rate)
        
        if category == 2: make_datacard_2tag(outDataCardsDir, modelName, signal_rate, norm, bkg_rate, observation, \
                          bkg_unc, bkg_unc_name, sig_unc, sig_eff_name, 'a', prefix)
        else: make_datacard_2tag_3bins(outDataCardsDir, modelName, signal_rate, norm, bkg_rate, observation, \
                              bkg_unc, bkg_unc_name, sig_unc, sig_eff_name, 'a', prefix)


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '25', '30', '35', '40', '45', '50', '55', '60', '65', '70', '75', '80', '85', '90', '95', '100', '125', '150', '175', '200', '225', '250', '275', '300', '325', '350', '375', '400', '425', '450', '475', '500', '525', '550', '575', '600', '625', '650', '675', '700', '725', '750', '775', '800', '825', '850', '875', '900', '925', '950', '975', '1000', '1250', '1500', '1750', '2000', '2250', '2500', '2750', '3000', '3250', '3500', '3750', '4000', '4250', '4500', '4750', '5000', '5250', '5500', '5750', '6000', '6250', '6500', '6750', '7000', '7250', '7500', '7750', '8000', '8250', '8500', '8750', '9000', '9250', '9500', '9750', '10000', '12500', '15000', '17500', '20000', '22500', '25000', '27500', '30000', '32500', '35000', '37500', '40000', '42500', '45000', '47500', '50000', '52500', '55000', '57500', '60000', '62500', '65000', '67500', '70000', '72500', '75000', '77500', '80000

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:251: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:251: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:251: RuntimeWarning: overflow encountered in square


CPU times: user 2min 8s, sys: 6.15 s, total: 2min 14s
Wall time: 5min 42s


In [8]:
k = 'data'
N_RECHIT_CUT = 80
a = np.count_nonzero(np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT))#bin A
b = np.logical_and(nhits1[k]>=N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT)#bin A
b = np.count_nonzero(np.logical_or(b, np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]>=N_RECHIT_CUT)))
c = np.count_nonzero(np.logical_and(nhits1[k]<N_RECHIT_CUT, nhits2[k]<N_RECHIT_CUT))
bkg_rate = np.array([b**2/4.0/c, b, c])
observation = [a,b,c]
print(bkg_rate)
print(observation)

[0.08333333 1.         3.        ]
[0, 1, 3]
